In [1]:
import sys
import os
import time
import torch
from tqdm import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

save_dir = '../data/'
train_file = 'episodes/train_set.pt'
val_file = 'episodes/val_set.pt'
test_file = 'episodes/test_set.pt'

# Define the features of the full data
device = 'cpu'
num_actions = 25
dim_subactions = 10
state_dim = 47
num_obs = 33
num_dem = 5
num_acuity_scores = 3
horizon = 21   # 1 more than the max horizon for masking purposes
action_temp = torch.eye(num_actions)

In [2]:
# This assumes MIMIC-III sepsis cohort has been extracted from https://github.com/microsoft/mimic_sepsis
full_data_file = os.path.join(save_dir, 'sepsis_final_data_withTimes_newActions.csv')
acuity_scores_file = os.path.join(save_dir, 'acuity_scores.csv')  # These are extracted using derive_acuities.py

full_zs = pd.read_csv(full_data_file)
acuity_scores = pd.read_csv(acuity_scores_file)

## Determine the train, val, test split (70/15/15), stratified by patient outcome
temp = full_zs.groupby('traj')['r:reward'].sum()
y = temp.values
X = temp.index.values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5, random_state=42)

# Drop unneeded meta features
full_zs = full_zs.drop(['m:presumed_onset', 'm:charttime'], axis=1)
full_zs['m:icustayid'] = full_zs['m:icustayid'].astype(int)

train_data = full_zs[full_zs['traj'].isin(X_train)].copy()
train_acuity = acuity_scores[acuity_scores['traj'].isin(X_train)]
train_trajectories = train_data['traj'].unique()

val_data = full_zs[full_zs['traj'].isin(X_val)]
val_acuity = acuity_scores[acuity_scores['traj'].isin(X_val)]
val_trajectories = val_data['traj'].unique()

test_data = full_zs[full_zs['traj'].isin(X_test)]
test_acuity = acuity_scores[acuity_scores['traj'].isin(X_test)]
test_trajectories = test_data['traj'].unique()

In [3]:
full_zs['step'].max()

19

In [4]:
full_zs

,traj,step,m:icustayid,o:gender,o:mechvent,o:re_admission,o:age,o:Weight_kg,o:GCS,o:HR,...,o:output_total,o:output_4hourly,r:reward,o:max_dose_vaso,o:input_4hourly,a:vaso,a:iv_OLD,a:iv,a:action_OLD,a:action
0,1,0,200003,-0.5,-0.5,-0.5,-0.975759,-0.178920,0.701549,-0.745922,...,0.778741,0.686365,0.0,0.0,50.0,0,2,1,10,5
1,1,1,200003,-0.5,-0.5,-0.5,-0.975759,-0.178920,0.701549,-1.118910,...,0.786546,0.578216,0.0,0.0,50.0,0,2,1,10,5
2,1,2,200003,-0.5,-0.5,-0.5,-0.975759,-0.178920,0.701549,-1.007298,...,0.792397,0.497215,0.0,0.0,50.0,0,2,1,10,5
3,1,3,200003,-0.5,-0.5,-0.5,-0.975759,-0.145806,0.701549,-0.540921,...,0.799349,0.556536,0.0,0.0,50.0,0,2,1,10,5
4,1,4,200003,-0.5,-0.5,-0.5,-0.975759,-0.137527,0.701549,-0.600713,...,0.810064,0.697897,0.0,0.0,50.0,0,2,1,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256568,19626,8,299995,-0.5,-0.5,-0.5,-2.469086,-1.408290,0.701549,0.221426,...,0.462327,0.730094,0.0,0.0,0.0,0,0,0,0,0
256569,19626,9,299995,-0.5,-0.5,-0.5,-2.469086,-1.408290,0.701549,-0.047638,...,0.503623,0.777200,0.0,0.0,0.0,0,0,0,0,0
256570,19626,10,299995,-0.5,-0.5,-0.5,-2.469086,-1.006779,0.701549,-0.182170,...,0.503623,-1.928841,0.0,0.0,0.0,0,0,0,0,0
256571,19626,11,299995,-0.5,-0.5,-0.5,-2.469086,-1.006779,0.701549,-1.093997,...,0.525883,0.621103,0.0,0.0,0.0,0,0,0,0,0


In [5]:
acuity_scores

,traj,step,c:SOFA,c:OASIS,c:SAPSii
0,1,0,3.0,16.0,14.0
1,1,1,5.0,16.0,24.0
2,1,2,5.0,15.0,22.0
3,1,3,6.0,15.0,22.0
4,1,4,3.0,16.0,15.0
...,...,...,...,...,...
256568,19626,8,0.0,13.0,7.0
256569,19626,9,2.0,8.0,11.0
256570,19626,10,6.0,12.0,15.0
256571,19626,11,2.0,12.0,17.0


In [6]:
print('Train:', len(y_train), y_train.mean())
print('Val  :', len(y_val), y_val.mean())
print('Test :', len(y_test), y_test.mean())

Train: 13738 0.8079778715970302
Val  : 2944 0.8077445652173914
Test : 2944 0.8077445652173914


In [7]:
################################################################
#          FORMAT DATA FOR USE IN SEQUENTIAL MODELS
################################################################

icustayid_col = 'm:icustayid'
dem_keep_cols = ['o:gender', 'o:mechvent', 'o:re_admission', 'o:age', 'o:Weight_kg']
obs_keep_cols = ['o:GCS', 'o:HR', 'o:SysBP',
       'o:MeanBP', 'o:DiaBP', 'o:RR', 'o:Temp_C', 'o:FiO2_1', 'o:Potassium',
       'o:Sodium', 'o:Chloride', 'o:Glucose', 'o:Magnesium', 'o:Calcium',
       'o:Hb', 'o:WBC_count', 'o:Platelets_count', 'o:PTT', 'o:PT',
       'o:Arterial_pH', 'o:paO2', 'o:paCO2', 'o:Arterial_BE', 'o:HCO3',
       'o:Arterial_lactate','o:PaO2_FiO2', 'o:SpO2', 'o:BUN', 'o:Creatinine',
       'o:SGOT', 'o:SGPT', 'o:Total_bili', 'o:INR']

dem_cols = [i for i in train_data.columns if i in dem_keep_cols]
obs_cols = [i for i in train_data.columns if i in obs_keep_cols]
ac_cols  = [i for i in train_data.columns if i[:2] == 'a:']
rew_cols = [i for i in train_data.columns if i[:2] == 'r:']
acuity_cols = [i for i in train_acuity.columns if i[:2] == 'c:']
#Assuming discrete actions and scalar rewards:

assert len(obs_cols) > 0, 'No observations present, or observation columns not prefixed with "o:"'
assert len(ac_cols) > 0, 'No actions present, or actions column not prefixed with "a:"'
assert len(rew_cols) > 0, 'No rewards present, or rewards column not prefixed with "r:"'
# assert len(ac_cols) == 1, 'Multiple action columns are present when a single action column is expected'
assert len(rew_cols) == 1, 'Multiple reward columns are present when a single reward column is expected'
# assert len(acuity_cols) == num_acuity_scores, 'Ensure that we have the right number of acuity scores'
ac_col = ac_cols[0]
rew_col = rew_cols[0]

## TRAINING DATA

In [8]:
df_data = train_data
df_acuity = train_acuity
trajectories = train_trajectories

In [9]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 13738/13738 [01:07<00:00, 204.78it/s]


In [10]:
# Eliminate single transition trajectories...
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [11]:
observations.shape

(13498, 21, 33)

In [12]:
torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(), 
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, train_file))

## Validation DATA

In [13]:
df_data = val_data
df_acuity = val_acuity
trajectories = val_trajectories

In [14]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 2944/2944 [00:14<00:00, 209.73it/s]


In [15]:
# Eliminate single transition trajectories...
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [16]:
observations.shape

(2895, 21, 33)

In [17]:
torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(),
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, val_file))

## Test DATA

In [18]:
df_data = test_data
df_acuity = test_acuity
trajectories = test_trajectories

In [19]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 2944/2944 [00:13<00:00, 210.47it/s]


In [20]:
# Eliminate single transition trajectories...
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [21]:
observations.shape

(2894, 21, 33)

In [22]:
torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(),
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, test_file))